In [2]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [58]:
data_path = os.path.join(os.getcwd(),'data')
files = os.listdir(data_path)[:-1]
files

['aisles.csv',
 'departments.csv',
 'orders.csv',
 'order_products__prior.csv',
 'order_products__train.csv',
 'products.csv']

In [60]:
os.path.join(data_path,files)

TypeError: join() argument must be str or bytes, not 'list'

In [65]:
aisles = pd.read_csv(os.path.join(data_path,files[0]), engine='pyarrow') # (134, 2)
departments = pd.read_csv(os.path.join(data_path,files[1]), engine='pyarrow') # (21, 2)
# orders = pd.read_csv(os.path.join(data_path,files[2]), engine='pyarrow') # (3421083, 7)
# orders_products_prior = pd.read_csv(os.path.join(data_path,files[3]),engine='pyarrow') # (32434489, 4)
# orders_products_train = pd.read_csv(os.path.join(data_path,files[4]),engine='pyarrow') # (1384617, 4)
products = pd.read_csv(os.path.join(data_path,files[5]),engine='pyarrow') # (49688, 4)

* aisles - 통로(코너, 냉동 department의 어류 이런 세부종류?) aisleid, aisle
* departments - department종류(냉동식품 등)  department id, department
* orders - userid, eval_set(prior, train, test), 주문번호, 주문요일?, 주문시간대, since prior order?
* order_products_prior - orderid, 상품id, add to cart order, reorderd(재주문 여부, none은 0으로)
* order_products_train - 
* products - 상품id, 상품명, aisleid, departmentid

# 결측치 여부

In [70]:
tables = [aisles, departments, orders, orders_products_prior, orders_products_train, products]

In [72]:
for t in tables:
    print(t.isnull().sum())

aisle_id    0
aisle       0
dtype: int64
department_id    0
department       0
dtype: int64
order_id                       0
user_id                        0
eval_set                       0
order_number                   0
order_dow                      0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64
order_id             0
product_id           0
add_to_cart_order    0
reordered            0
dtype: int64
order_id             0
product_id           0
add_to_cart_order    0
reordered            0
dtype: int64
product_id       0
product_name     0
aisle_id         0
department_id    0
dtype: int64


days_since_prior_order에만 결측치 존재

In [79]:
prior = orders[orders['eval_set']== 'prior']
train = orders[orders['eval_set'] == 'train']
test = orders[orders['eval_set'] == 'test']

In [80]:
print(prior.shape)
print(train.shape)
print(test.shape)
print(orders.shape)


(3214874, 7)
(131209, 7)
(75000, 7)
(3421083, 7)


In [84]:
del train['eval_set'], test['eval_set'], prior['eval_set']

In [85]:
train.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
10,1187899,1,11,4,8,14.0
25,1492625,2,15,1,11,30.0
49,2196797,5,5,0,11,6.0
74,525192,7,21,2,11,6.0
78,880375,8,4,1,14,10.0


In [88]:
pd.merge(orders_products_train, train, how='inner')

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0
...,...,...,...,...
32434484,3421083,39678,6,1
32434485,3421083,11352,7,0
32434486,3421083,4600,8,0
32434487,3421083,24852,9,1


In [92]:
orders_products_train.shape, train.shape

((1384617, 4), (131209, 6))

In [104]:
temp = orders_products_train.set_index('order_id')
orders_train = pd.merge(temp, train, on='order_id')
orders_train.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1,49302,1,1,112108,4,4,10,9.0
1,1,11109,2,1,112108,4,4,10,9.0
2,1,10246,3,0,112108,4,4,10,9.0
3,1,49683,4,0,112108,4,4,10,9.0
4,1,43633,5,1,112108,4,4,10,9.0


In [107]:
temp = orders_products_prior.set_index('order_id')
orders_prior = pd.merge(temp, prior, on='order_id')
orders_prior.head() # 1분걸림

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,33120,1,1,202279,3,5,9,8.0
1,2,28985,2,1,202279,3,5,9,8.0
2,2,9327,3,0,202279,3,5,9,8.0
3,2,45918,4,1,202279,3,5,9,8.0
4,2,30035,5,0,202279,3,5,9,8.0


In [116]:
total_orders = pd.concat([orders_prior, orders_train])
total_orders.to_csv('total_orders.parquet', index=False)

orders, orders_products_prior, orders_products_train을 모두 합쳐줬다.(test 제외)

In [140]:
# total_orders = pd.read_csv('total_orders.csv',engine='pyarrow') # 1분 15초

In [7]:
total_orders = pd.read_parquet('total_orders.parquet',engine='fastparquet') # 5초

In [9]:
del total_orders

In [10]:
practice = pd.read_parquet('practice.parquet',engine='fastparquet') # 연습용 100만개

In [11]:
practice

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,33120,1,1,202279,3,5,9,8.0
1,2,28985,2,1,202279,3,5,9,8.0
2,2,9327,3,0,202279,3,5,9,8.0
3,2,45918,4,1,202279,3,5,9,8.0
4,2,30035,5,0,202279,3,5,9,8.0
...,...,...,...,...,...,...,...,...,...
1000104,105641,12080,4,1,143162,5,4,18,3.0
1000105,105641,38890,5,1,143162,5,4,18,3.0
1000106,105641,38003,6,0,143162,5,4,18,3.0
1000107,105641,13885,7,0,143162,5,4,18,3.0


In [23]:
pd.options.display.max_rows = 10
practice.set_index('user_id','order_id')[:100]

C:\Users\이정진\AppData\Local\Temp\ipykernel_15256\657486680.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.set_index except for the argument 'keys' will be keyword-only.
  practice.set_index('user_id','order_id')[:100]


,order_id,product_id,add_to_cart_order,reordered,order_number,order_dow,order_hour_of_day,days_since_prior_order
user_id,,,,,,,,
202279,2,33120,1,1,3,5,9,8.0
202279,2,28985,2,1,3,5,9,8.0
202279,2,9327,3,0,3,5,9,8.0
202279,2,45918,4,1,3,5,9,8.0
202279,2,30035,5,0,3,5,9,8.0
...,...,...,...,...,...,...,...,...
143742,11,1313,4,1,4,1,19,23.0
143742,11,31506,5,1,4,1,19,23.0
152610,12,30597,1,1,22,6,8,10.0


In [41]:
df = practice[['order_id','product_id','user_id']]

In [47]:
df2 = df.groupby(['user_id','order_id']).product_id.apply(list).reset_index(name='order_list')
df2

,user_id,order_id,order_list
0,10,68288,"[46979, 47380, 20995, 43014, 15011, 27156, 135..."
1,10,83395,"[30489, 47591, 24184, 260, 9339, 38293, 17828,..."
2,1000,18216,"[30563, 28465, 14870, 30492, 26165, 49683, 196..."
3,10000,12156,"[11481, 27423, 24518, 21167, 38578, 5077, 1695..."
4,10000,103998,"[41570, 7175, 33000, 5077, 42356, 9957, 22935,..."
...,...,...,...
99265,99985,44451,"[3896, 3721, 24852, 15693, 27845, 10912, 21137..."
99266,99986,15981,"[47626, 9076, 48778, 34234, 39693, 38996, 1664..."
99267,99987,95180,"[27845, 6104, 29487, 19660, 41665, 22035, 8575..."
99268,99993,63645,"[43631, 24852, 39170, 40489, 25814, 28521, 23413]"


In [75]:
df2['order_list'][0]

[46979,
 47380,
 20995,
 43014,
 15011,
 27156,
 13512,
 40604,
 13198,
 9339,
 5646,
 28986,
 27104,
 41220,
 22825,
 34358,
 44910,
 15290,
 14992,
 36695,
 47042,
 23165,
 15937,
 25931,
 5818,
 1529,
 32537,
 11782,
 30489,
 36735,
 18441,
 15392,
 7632,
 32299,
 48204,
 28535,
 34128,
 8988,
 13212,
 7746,
 35973,
 35725,
 45664,
 16857,
 31506,
 47526]

In [73]:
len([0]*49688)

49688

In [71]:
pd.DataFrame(columns = products['product_id'].unique())

,1,2,3,4,5,6,7,8,9,10,...,49679,49680,49681,49682,49683,49684,49685,49686,49687,49688
